In [1]:
import numpy as np
import tonic
import tonic.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader
#from tonic.dataset import Dataset
from typing import Callable, Optional
import torch
import matplotlib.pyplot as plt
from torchvision.ops import masks_to_boxes
from torchvision.utils import draw_bounding_boxes
import matplotlib.patches as patches

In [2]:
import sys
sys.path.append("./mnist_sg_cnn")

In [3]:
import utils
import snn_utils
import base_model
import lenet_decolle_model

In [4]:
dir = "/media/user/EVIMO/raw/imo/eval/scene15_dyn_test_01/left_camera/ground_truth_000000"
sensor_size = [640, 480, 2]
batch_size = 4
framerate = 200
device = 'cpu'

In [5]:
# class EVIMO(Dataset):
#     def __init__(self,
#                  data: np.array,
#                  masks: list,
#                  transform: Optional[Callable] = None
#                 ):
#         #super().__init__(save_to, transform=transform)
#         self.masks = masks
#         if transform is not None:
#             self.data = transform(data)
#             if len(self.data) != len(self.masks):
#                 raise ValueError("Number of frames after transforms does not match number of masks")
#         else:
#             self.data = data

#         #target = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#         self.targets = []
        
#         num_masks_include = 0
#         for mask in self.masks:
#             if 23 in mask:
#                 num_masks_include += 1
#                 self.targets.append([0, 1])
#             else:
#                 self.targets.append([1, 0])

#         print(f"Target present in: {num_masks_include} of {len(self.masks)} masks. {num_masks_include/len(self.masks)}")
            

#         self.targets = np.asarray(self.targets)
        
#         #self.targets = torch.from_numpy(np.array(len(self.data)*[target]))
        

#     def __getitem__(self, index):
#         #return self.data[index], self.targets[index]
#         return {"data" : self.data[index], "mask": self.masks[index], "target": self.targets[index]}
#     def __len__(self) -> int: 
#         return len(self.data)
    

In [6]:
class EVIMO(Dataset):
    def __init__(self,
                 dir: str,
                 length: int,
                ):
        self.dir = dir
        self.length = length

        self.labels = np.load(self.dir + "/labels.npy")

    def __getitem__(self, index):
        return {"data": np.load(self.dir + "/" + str(index) + ".npy"), "target": self.labels[index]}

    def __len__(self) -> int:
        return self.length
    
    

In [7]:
# # Denoise removes isolated, one-off events
# # time_window
# frame_transform = transforms.Compose([#transforms.Denoise(filter_time=10000),
#                                       transforms.ToFrame(sensor_size=sensor_size,
#                                                          time_window=5000)
#                                      ])

In [8]:
tonic_dataset = EVIMO(dir="./data/EVIMO/train", length=703)

In [9]:
1-0.2972972972972973

0.7027027027027026

In [10]:
def plot_frame(frame, mask): # TODO: Put the mask processing into bounding box into the tonic dataset?
    print(mask)
    img = frame[1] - frame[0]
    fig, ax = plt.subplots(1,2, figsize=(10,10))
    ax[0].imshow(img)

    mask = torch.from_numpy(mask)
    obj_ids = torch.unique(mask)
    #print(obj_ids)
    obj_ids = obj_ids[1:]
    print(obj_ids)
    detection_masks = mask == obj_ids[:, None, None]
    #print(detection_masks)
    
    boxes = masks_to_boxes(detection_masks)
    print(boxes)
    #for box in boxes:
    box = boxes[5]
    ax[0].add_patch(patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=1, edgecolor='r', facecolor='none'))


    ax[1].imshow(detection_masks[5])
    print(f"obj 23 in mask: {23 in mask}")
    
    ax[0].axis("off")
    ax[1].axis("off")
    plt.show()


#item = tonic_dataset.__getitem__(600)

#print(binned_events.shape)
#plot_frame(item['data'], item['mask'])

In [11]:
batched_trainloader = DataLoader(tonic_dataset, batch_size=batch_size, shuffle=True) # collate_fn=tonic.collation.PadTensors(),
single_trainloader = DataLoader(tonic_dataset, batch_size=1, shuffle=False)

In [12]:
#for i_batch, sample_batched in enumerate(trainloader):
#    print(i_batch, sample_batched['data'].size(), sample_batched['target'].size())

In [13]:
datait = iter(batched_trainloader)
batched_data = next(datait)

In [14]:
#batched_data['data']

In [15]:
batched_data['target'].shape

torch.Size([4, 2])

In [16]:
item = next(iter(batched_trainloader))
data = item['data']
target = item['target']

def decolle_loss(r, s, tgt):
    loss_tv = 0
    for i in range(len(r)):
        #print(f"Loss Readout shape : {r[i].shape}")
        #print(f"Loss Target shape : {tgt.shape}")
        loss_tv += loss(r[i],tgt) 
    return loss_tv

loss = torch.nn.SmoothL1Loss()

convnet_sg = lenet_decolle_model.LenetDECOLLE(out_channels=2,
                    Nhid=[32, 64, 128, 256], #Number of convolution channels
                    Mhid=[64, 256, 512],
                    kernel_size=[4, 16, 32, 64],
                    pool_size=[2, 4, 4, 8],
                    input_shape=[2, 480, 640],  # data.shape[1:],
                    alpha=[.95],
                    alpharp=[.65],
                    beta=[.92],
                    num_conv_layers=4,
                    num_mlp_layers=3,
                    lc_ampl=0.5).to(device)

#net = lenet_decolle_model.LenetDECOLLE(Nhid=[1,8],Mhid=[32,64],out_channels=2, input_shape=[2, 480, 640]).to(device)

#convnet_sg

data_d = data.to(device)
target_d = target.to(device)
convnet_sg.init_parameters_evimo(data_d) # Modifies readout dim

LAYER SIZE: 2439712
STDV: 0.00032011109219189804
LAYER SIZE: 298304
STDV: 0.0009154623036203756
LAYER SIZE: 34048
STDV: 0.002709718654762875
LAYER SIZE: 512
STDV: 0.022097086912079608
LAYER SIZE: 64
STDV: 0.0625
LAYER SIZE: 256
STDV: 0.03125
LAYER SIZE: 512
STDV: 0.022097086912079608


In [17]:
data_d.shape

torch.Size([4, 2, 480, 640])

In [18]:
data_d.transpose(0, 1).shape

torch.Size([2, 4, 480, 640])

In [19]:
data.shape[1:]

torch.Size([2, 480, 640])

In [20]:
target_d

tensor([[0, 1],
        [0, 1],
        [1, 0],
        [1, 0]])

In [21]:
target_d.shape

torch.Size([4, 2])

In [22]:
opt_conv = torch.optim.Adamax(convnet_sg.get_trainable_parameters(), lr=1e-4, betas=[0., .95])
#convnet_sg.requires_init = False
for e in range(5):        
    error = []
    accuracy=[]
    for item in iter(batched_trainloader):
        data = item['data']
        label = item['target']

        if label.shape[0] != 4:
            #print(f"last entry")
            break

        
        
        convnet_sg.train()
        loss_hist = 0
        data_d = data.to(device)
        label_d = label.to(device)
        convnet_sg.init_evimo(data_d)
        readout = 0
        
        #for i in range(0, len(data_d)):
            #d = data_d[i:i+1]
            #d = torch.zeros([1, 2, 480, 640])
            #print(f"Data shape: {d.shape}")
            
            #print(f"Target shape: {label_d[i].shape}")
            #print(label_d[i])
        st, rt, ut = convnet_sg.forward(data_d)        
        loss_tv = decolle_loss(rt, st, label_d)
        
        loss_tv.backward()
        opt_conv.step()
        opt_conv.zero_grad()
        loss_hist += loss_tv
        readout += rt[-1]

        #print(f"Readout: {readout}")
        error += (readout.argmax(axis=1)!=label_d.argmax(axis=1)).float()
        accuracy+=(readout.argmax(axis=1)==label_d.argmax(axis=1)).float()
        
    print('Training Error', torch.mean(torch.Tensor(error)).data)
    print('Training accuracy', torch.mean(torch.Tensor(accuracy)).data)     
    print('Epoch', e, 'Loss', loss_hist.data)
    PATH = './EVIMO_class_cnn.pth'  
    torch.save(convnet_sg.state_dict(), PATH)

Training Error tensor(0.5700)
Training accuracy tensor(0.4300)
Epoch 0 Loss tensor(1.1434)
Training Error tensor(0.5414)
Training accuracy tensor(0.4586)
Epoch 1 Loss tensor(0.9750)


KeyboardInterrupt: 

In [ ]:
type(loss_tv)

In [ ]:
error = []
accuracy=[]
y_pred = []
y_true = []
#convnet_sg.requires_init = True
for item in iter(batched_trainloader):
    #print("Item")
    #print(item)
    data = item['data']
    label = item['target']
    if label.shape[0] != 4:
        #print(f"last entry: {label}")
        break

    #print("earliest label")
    #print(label)
    #print(label.shape)

    
    loss_hist = 0
    data_d = data.to(device)
    label_d = label.to(device)
    print(f"data_d shape: {data_d.shape}")
    print(f"data_d_tranpose shape: {data_d.transpose(0,1).shape}")
    print(f"Len convnet: {len(convnet_sg)}")
    print(f"data init shape: {data_d.transpose(0,1)[:, 0, :, :].shape}")
    print(f"Test data shape: {data_d[0].shape}")
    
    convnet_sg.init_evimo(data_d)
    readout = 0
    with torch.no_grad():
        #for i in range(0, len(data_d)):
        #d = data_d[i:i+1]
        
        #print(d.shape)
        
        st, rt, ut = convnet_sg.forward(data_d)

        
        
        print(f"Label shape: {label_d.shape}")
        print("Readout")
        print(rt)
        print(rt[0].shape)

        #print("labels")
        #print(label_d)
        #print(label_d.shape)
        
        
        loss_tv = decolle_loss(rt, st, label_d)
        
        
        loss_hist += loss_tv
        readout += rt[-1]
        print(f"Readout: {readout}")
        output = (readout.argmax(axis=1)).data.cpu().numpy()
        print(f"Output: {output}")
        y_pred.extend(output)
        #labels = label_d.argmax(axis=1)
        print(f"Labels: {label_d}")
        labels = (label_d.argmax(axis=1)).data.cpu().numpy()
        print(f"Labels: {labels}")
        y_true.extend(labels)
        accuracy+=(readout.argmax(axis=1)==label_d.argmax(axis=1)).float()
        error += (readout.argmax(axis=1)!=label_d.argmax(axis=1)).float()
        break
        
print('Testing Error', torch.mean(torch.Tensor(error)).data)
print('Testing accuracy', torch.mean(torch.Tensor(accuracy)).data)

In [ ]:
print(y_true)
print(y_pred)

In [ ]:
"""
convnet_sg = lenet_decolle_model.LenetDECOLLE( out_channels=2,
                    Nhid=[16,32], #Number of convolution channels
                    Mhid=[64],
                    kernel_size=[7],
                    pool_size=[2,2],
                    input_shape=[2, 480, 640],  # data.shape[1:],
                    alpha=[.95],
                    alpharp=[.65],
                    beta=[.92],
                    num_conv_layers=2,
                    num_mlp_layers=1,
                    lc_ampl=.5).to(device)

"""

In [ ]:
net = lenet_decolle_model.LenetDECOLLE(Nhid=[16,32],Mhid=[64],out_channels=2, pool_size=[2,2], num_conv_layers=2, num_mlp_layers=1, lc_ampl=.5, input_shape=[2, 480, 640])
d = torch.zeros([1, 2, 480, 640])
print(d.shape)
st, rt, ut = net.forward(d)
print(rt[0].shape)

In [ ]:
print(rt)

In [ ]:
target = torch.zeros([1, 2])
target

In [ ]:
loss_tv = decolle_loss(rt, st, target)